In [1]:
import os
from os.path import join
from io import StringIO
from glob import glob
from textwrap import dedent

import sympy
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from collections import defaultdict

import QDYN
from src.single_sided_network_v1 import network_slh
from src.dicke_single_model_v2 import write_dicke_single_model, err_dicke_single
from src.doit_actions_v2 import run_traj_prop

from doit.tools import register_doit_as_IPython_magic
import clusterjob

In [2]:
DOIT_CONFIG = {
    'backend': 'json',
    'dep_file': '.doit_db/2017-10-04_mg_method_comparison_dicke.json',
}
register_doit_as_IPython_magic()

In [3]:
clusterjob.JobScript.read_defaults('./config/mlhpc_cluster.ini')

In [4]:
! qdyn_version

QDYN 2.0dev revision 36f75756deefe59f55ee5dec363429c8cf4f994f (master)
  features: no-check-cheby, no-check-newton, no-parallel-ham, use-mkl=sequential, use-mpi=intel, parallel-oct, backtraces, no-debug, no-no-ipo
  compiled with ifort on Tue Oct 10 08:56:03 2017 on host mlhpc2


$
\newcommand{ket}[1]{\vert #1 \rangle}
\newcommand{bra}[1]{\langle #1 \vert}
\newcommand{Op}[1]{\hat{#1}}
$

# Compare variants of Krotov's method for optimization of a two-node Dicke state

## action wrappers

In [5]:
def update_config(rf, lambda_a, iter_stop, J_T_conv):
    config = QDYN.config.read_config_file(join(rf, 'config'))
    config['oct']['iter_stop'] = iter_stop
    config['oct']['J_T_conv'] = J_T_conv
    for pulse_config in config['pulse']:
        pulse_config['oct_lambda_a'] = lambda_a
    QDYN.config.write_config(config, join(rf, 'config'))

In [6]:
def submit_optimization(rf, n_trajs, variant, task):
    """Asynchronously run optimization, using either the
    standard QDYN implementation (variant = 'normal') or the 'crossoct' QDYN
    implementation at a particular revision (variant='cross')"""
    if os.path.isfile(join(rf, 'oct.job.dump')):
        return
    bodies = {
        'normal': dedent(r'''
    {module_load}
    '''),
        'cross': dedent(r'''
    {module_load}
    module load prefix/crossoct

    qdyn_version | grep 36f75756deef || {cancel_cmd}
    ''')}
    for key, body in bodies.items():
        body += dedent(r'''
    cd $CLUSTERJOB_WORKDIR
    pwd
    export MPICH_NO_BUFFER_ALIAS_CHECK=1
    export I_MPI_COMPATIBILITY=4
    OMP_NUM_THREADS=1 {runner} qdyn_optimize --n-trajs={n_trajs} \
         --J_T=J_T_sm .
    ''')
        bodies[key] = body
    body = bodies[variant]
    taskname = "oct_%s" % task.name.split(":")[-1]
    jobscript = clusterjob.JobScript(
        body=body, jobname=taskname, workdir=rf,
        stdout='oct.log')
    runner = {
        'copper': 'aprun -B',
        'mlhpc': 'mpirun -n %d' % n_trajs}
    if jobscript.cluster_name != 'copper':
        jobscript.resources['ppn'] = int(n_trajs)
        if n_trajs == 1:
            runner['mlhpc'] = ''
    jobscript.runner = runner[jobscript.cluster_name]
    jobscript.n_trajs = str(int(n_trajs))
    run = jobscript.submit(cache_id=taskname)
    run.dump(join(rf, 'oct.job.dump'))

In [7]:
def wait_for_clusterjob(dumpfile):
    """Wait until the clusterjob.AsyncResult cached in the given dumpfile ends"""
    try:
        run = clusterjob.AsyncResult.load(dumpfile)
        run.wait()
        os.unlink(dumpfile)
        return run.successful()
    except OSError:
        pass

In [8]:
def write_rho_prop_custom_config(rf, oct_iter, config_out):
    """Write `config_out` in `rf` for propagation of the optimized pulses
    ``<oct_outfile>.<oct_iter>``, based on the template in ``rf/config_rho``
    """
    config = QDYN.config.read_config_file(join(rf, 'config_rho'))
    if 'oct' in config:
        # we wouldn't want to accidentally use this config file for OCT
        del config['oct']
    for pulse_config in config['pulse']:
        pulse_file = "%s.%08d" % (pulse_config['oct_outfile'], int(oct_iter))
        # we wouldn't want to accidentally propagate the final optimized pulse
        if 'oct_outfile' in pulse_config:
            del pulse_config['oct_outfile']
        assert os.path.isfile(join(rf, pulse_file))
        pulse_config['filename'] = pulse_file
    for obs_config in config['observables']:
        outfile = obs_config['outfile']
        if outfile == 'P_target.dat':
            obs_config['outfile'] = "%s.%08d" % (outfile, int(oct_iter))
            obs_config['from_time_index'] = -1
            obs_config['to_time_index'] = -1
            config['observables'] = [obs_config, ]
            break
    for key in ['write_jump_record', 'write_final_state']:
        if key in config['user_strings']:
            del config['user_strings'][key]
    assert len(config['observables']) == 1
    assert config['user_logicals']['rho']
    QDYN.config.write_config(config, join(rf, config_out))

In [9]:
def sort_key_oct_iters(f):
    """Key for sorting files according to iteration number (file extension)"""
    try:
        a, b = f.rsplit('.', maxsplit=1)
        return (a, int(b))
    except ValueError:
        return (f, 0)

In [10]:
def collect_rho_prop_errors(*P_target_obs_files, outfile):
    """Combine the expectation values for the target projector for different
    OCT iterations into a single data file (`outfile`)"""
    T = None
    with open(outfile, 'w') as out_fh:
        out_fh.write("#%9s%25s\n" % ('iter', 'Error 1-<P>_rho'))
        for obs_file in sorted(P_target_obs_files, key=sort_key_oct_iters):
            n_iter = int(obs_file.rsplit('.', maxsplit=1)[-1])
            data = np.genfromtxt(obs_file)
            assert data.shape == (3, )
            if T is None:
                T = data[0]
                # we want to make sure all files have the same final time
            assert abs(data[0] - T) < 1e-14
            out_fh.write("%10d%25.16E\n" % (n_iter, 1.0 - data[1]))

## custom uptodate routines

In [11]:
from src.qdyn_model_v2 import pulses_uptodate

## task definitions

In [12]:
def runfolder(row):
    int_part, frac_part = ("%.2f" % row['T']).split('.')
    root = './data/method_comparison_dicke/'
    if frac_part == '00':
        T_str = "%s" % int_part
    else:
        T_str = '%s_%s' % (int_part, frac_part)
    rf = "T%s_%s_ntrajs%s" % (T_str, row['variant'], row['n_trajs'])
    return join(root, rf)

In [13]:
def task_create_runfolder():
    """Create all necessary runfolders for the runs defined in params_df"""
    jobs = {}
    slh_models = {}
    n_nodes = 2
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        try:
            slh = slh_models[n_nodes]
        except KeyError:
            slh = network_slh(
                n_cavity=2, n_nodes=n_nodes, topology='driven_bs_fb')
            slh_models[n_nodes] = slh
        if rf in jobs:
            continue
        mcwf = {  #   variant => whether to use MCWF
            'rho': False, 'nonherm': False, 'independent': True,
            'cross': True}
        jobs[rf] = {
            'name': str(rf),
            'actions': [
                # write the density matrix propagation data
                (write_dicke_single_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=500,
                    kappa=1.0, E0_cycles=2,
                    mcwf=False, non_herm=False, # this defines rho-prop
                    config='config_rho')),
                # write the data for the optimization
                (write_dicke_single_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=500,
                    kappa=1.0, E0_cycles=2, mcwf=mcwf[row['variant']],
                    non_herm=(row['variant'] != 'rho'),
                    lambda_a=row['lambda_a'], keep_pulses='all',
                    iter_stop=int(row['iter_stop']), J_T_conv=row['J_T_conv'],
                    config='config'))],
            'targets': [join(rf, 'config_rho'), join(rf, 'config')],
            'uptodate': [True, ] # up to date if targets exist
        }
    for job in jobs.values():
        yield job

In [14]:
def task_update_runfolder():
    """For every row in params_df, update the config file in the appropriate
    runfolder with the value in that row
    (adjusting lambda_a, iter_stop and J_T_conv only)"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        # we only update the config after any earlier optimization has finished
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        rf_jobs[rf].append(ind)
        yield {
            'name': str(ind),
            'actions': [
                (update_config, [], dict(
                    rf=rf, lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop']),
                    J_T_conv=row['J_T_conv']))],
            'file_dep': [join(rf, 'config')],
            'uptodate': [False, ],  # always run task
            'task_dep': task_dep}


In [15]:
def task_submit_optimization():
    """Run optimization for every runfolder from params_df"""
    rf_jobs = defaultdict(list)
    optimization_variant = {
        'rho': 'normal',
        'nonherm': 'normal',
        'independent': 'normal',
        'cross': 'cross',
    }
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        task_dep.append('update_runfolder:%s' % ind)
        yield {
            'name': str(ind),
            'actions': [
                (submit_optimization, [rf, ],
                 dict(n_trajs=row['n_trajs'],
                      variant=optimization_variant[row['variant']]))],
                # 'task' keyword arg is added automatically
            'task_dep': task_dep,
            'uptodate': [(pulses_uptodate, [], {'rf': rf}), ],
        }

In [16]:
def task_wait_for_optimization():
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        yield {
            'name': str(ind),
            'task_dep': ['submit_optimization:%d' % ind],
            'actions': [
                (wait_for_clusterjob, [join(rf, 'oct.job.dump')], {}),]}

Additional tasks for evaluating the optimization success through a propagation in Liouville space are defined farther below

##  OCT Submission

In [17]:
params_data_str = r'''
# T  lambda_a      variant  n_trajs   iter_stop   J_T_conv
  5     0.001          rho        1        5000       1e-4
  5     0.001      nonherm        1        5000       1e-8
  5     0.001  independent        1        5000       1e-8
  5     0.001  independent        2        5000       1e-8
  5     0.001  independent        4        5000       1e-8
  5     0.001  independent        8        5000       1e-8
  5     0.001  independent       16        5000       1e-8
  5     0.001  independent       32        5000       1e-8
  5     0.001  independent       64        5000       1e-8
  5     0.001  cross              2        5000       1e-8
'''
params_df = pd.read_fwf(
        StringIO(params_data_str), comment='#', header=1,
        names=['T', 'lambda_a', 'variant', 'n_trajs', 'iter_stop', 'J_T_conv'],
        converters={
            'T': float, 'lambda_a': float, 'variant': str,
            'n_trajs': int, 'iter_stop': int, 'J_T_conv': float})

In [18]:
print(params_df.to_string())

     T  lambda_a      variant  n_trajs  iter_stop      J_T_conv
0  5.0     0.001          rho        1       5000  1.000000e-04
1  5.0     0.001      nonherm        1       5000  1.000000e-08
2  5.0     0.001  independent        1       5000  1.000000e-08
3  5.0     0.001  independent        2       5000  1.000000e-08
4  5.0     0.001  independent        4       5000  1.000000e-08
5  5.0     0.001  independent        8       5000  1.000000e-08
6  5.0     0.001  independent       16       5000  1.000000e-08
7  5.0     0.001  independent       32       5000  1.000000e-08
8  5.0     0.001  independent       64       5000  1.000000e-08
9  5.0     0.001        cross        2       5000  1.000000e-08


In [19]:
import logging
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(
    level=logging.DEBUG, filename='./data/method_comparison_dicke.log')

In [20]:
%doit -n 20 wait_for_optimization

-- create_runfolder:./data/method_comparison_dicke/T5_rho_ntrajs1
-- create_runfolder:./data/method_comparison_dicke/T5_nonherm_ntrajs1
-- create_runfolder:./data/method_comparison_dicke/T5_independent_ntrajs1
-- create_runfolder:./data/method_comparison_dicke/T5_independent_ntrajs2
-- create_runfolder:./data/method_comparison_dicke/T5_independent_ntrajs4
-- create_runfolder:./data/method_comparison_dicke/T5_independent_ntrajs8
-- create_runfolder:./data/method_comparison_dicke/T5_independent_ntrajs16
-- create_runfolder:./data/method_comparison_dicke/T5_independent_ntrajs32
-- create_runfolder:./data/method_comparison_dicke/T5_independent_ntrajs64
-- create_runfolder:./data/method_comparison_dicke/T5_cross_ntrajs2
.  update_runfolder:0
.  update_runfolder:1
.  update_runfolder:2
.  update_runfolder:4
.  update_runfolder:3
.  update_runfolder:5
.  update_runfolder:6
-- submit_optimization:0
.  update_runfolder:8
-- submit_optimization:2
.  update_runfolder:7
.  update_runfolder:9
-- su

## Evaluate error exactly from density matrix propagation

In [28]:
RHO_PROP_ITERS = np.arange(3000, step=10)

In [29]:
def task_rho_prop_create_custom_config():
    """For every optimized pulse, at intermediate iteration numbers,
    write a custom config file the will propagate this pulse"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        for oct_pulse_file in glob(join(rf, 'pulse1.oct.dat.0*')):
            oct_iter = int(os.path.splitext(oct_pulse_file)[-1][1:])
            if oct_iter not in RHO_PROP_ITERS:
                continue  # skip; we don't want to propagate *all* pulses
            assert os.path.isfile(oct_pulse_file.replace('pulse1', 'pulse2'))
            config_out = 'config_rho.%08d' % oct_iter
            yield {
                'name': str(rf) + "/%s" % config_out,
                'actions': [
                    (write_rho_prop_custom_config,
                     [rf, oct_iter, config_out])],
                'targets': [join(rf, config_out)],
                'uptodate': [True, ] # up to date if target exists
        }

In [30]:
def task_evaluate_rho_prop_error():
    """For every custom rho prop config file, perform the propagation"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        for custom_config in glob(join(rf, 'config_rho.0*')):
            oct_iter = int(os.path.splitext(custom_config)[-1][1:])
            yield {
                'name': str(rf) + "/%s" % custom_config,
                'actions': [
                    (run_traj_prop, [rf, ], dict(
                        n_trajs=1, n_procs=None, use_oct_pulses=False,
                        config=os.path.split(custom_config)[-1]))],
                'targets': [join(rf, 'P_target.dat.%08d' % oct_iter)],
                'file_dep': [custom_config],
                'uptodate': [True, ] # up to date if target exists
        }

In [31]:
def task_collect_rho_prop_errors():
    """For every custom rho prop config file, perform the propagation"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        P_target_obs_files = glob(join(rf, 'P_target.dat.0*'))
        target_file = join(rf, 'rho_prop_error.dat')
        yield {
            'name': target_file,
            'actions': [
                (collect_rho_prop_errors, P_target_obs_files,
                 dict(outfile=target_file))],
            'targets': [target_file],
            'file_dep': P_target_obs_files,
            'uptodate': [False, ]
            # We always create a new file
        }

In [32]:
%doit -n 20 rho_prop_create_custom_config

-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000350
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000420
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000450
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000570
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000880
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000240
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000660
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000720
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000060
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000130
-- rho_pro

.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001410
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001810
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000630
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000650
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001150
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002140
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001920
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002530
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000080
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000690
.  rho_pro

.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001660
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001530
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001340
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001940
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000550
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000820
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000010
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001560
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001070
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001020
.  rho_pro

.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001400
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001570
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000750
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000560
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002320
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002190
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001540
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002470
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001880
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001520
.  rho_pro

-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_nonherm_ntrajs1/config_rho.00000380
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_nonherm_ntrajs1/config_rho.00002480
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_nonherm_ntrajs1/config_rho.00000300
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_nonherm_ntrajs1/config_rho.00000020
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_nonherm_ntrajs1/config_rho.00000730
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_nonherm_ntrajs1/config_rho.00000070
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_nonherm_ntrajs1/config_rho.00000520
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_nonherm_ntrajs1/config_rho.00000580
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_nonherm_ntrajs1/config_rho.00001490
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_nonher

...

.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs64/config_rho.00002380
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs64/config_rho.00001550
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs64/config_rho.00001140
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs64/config_rho.00002920
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs64/config_rho.00002840
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs64/config_rho.00001990
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs64/config_rho.00002580
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs64/config_rho.00000150
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs64/config_rho.00001680
-- rho_prop_create_custom_co

.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002410
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001110
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001010
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001450
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001870
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002900
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000120
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000100
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000530
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho

.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002260
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002080
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002050
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000430
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000470
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000310
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000770
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002680
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000740
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho

.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001820
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001330
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002030
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002460
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002220
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002850
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002890
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000850
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000340
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho

.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002180
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001300
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002130
.  rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001770


In [33]:
%doit -n 20 evaluate_rho_prop_error

-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002800
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000540
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000990
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000630
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000830
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002350
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002600
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002780
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000240
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002400
-- rho_pro

-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00002680
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00002860
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00001920
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00002430
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00001500
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00001780
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00000030
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00001230
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00001350
-- rho_prop_create_custom_config:./da

-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00002560
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00002940
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00001900
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00000680
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00001330
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00000940
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00001750
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00002610
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00002710
-- rho_prop_create_custom_co

-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001260
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001450
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002020
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001730
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001120
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001030
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000910
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002890
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002070
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho

-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000970
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000960
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000050
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001380
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001860
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001820
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000730
.  evaluate_rho_prop_error:

...

-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000240
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000820
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002780
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002400
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001130
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000180
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001940

.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001590
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002340
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002680
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002860
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001920
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001780
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002430

-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000340
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001790
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001650
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001410
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002460
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002770
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002910

.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001560
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002930
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000800
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002820
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002980
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002520
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001580

-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000870
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00002180
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001970
.  evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001440


In [34]:
%doit -n 20 collect_rho_prop_errors

-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000900
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001180
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000870
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002120
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001900
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001010
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002420
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001580
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000510
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000940
-- rho_pro

-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00001610
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00002920
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00002540
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00002550
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00001490
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00002790
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00002090
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00001330
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00001830
-- rho_prop_create_custom_config:./da

-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00002000
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00000890
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00000200
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00001420
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00002160
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00001870
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00000790
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00001800
-- rho_prop_create_custom_config:./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00000630
-- rho_prop_create_custom_co

-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002700
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002680
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000240
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00002660
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001940
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00001310
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_rho_ntrajs1/./data/method_comparison_dicke/T5_rho_ntrajs1/config_rho.00000910
-- evaluate_rho_prop_error:

-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_independent_ntrajs2/./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00002230
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_independent_ntrajs2/./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00002390
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_independent_ntrajs2/./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00001940
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_independent_ntrajs2/./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00000490
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_independent_ntrajs2/./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00000360
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_independent_ntrajs2/./data/method_comparison_dicke/T5_independent_ntrajs2/config_rho.00001560
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_independe

-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_independent_ntrajs16/./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00002700
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_independent_ntrajs16/./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00002140
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_independent_ntrajs16/./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00000830
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_independent_ntrajs16/./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00002600
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_independent_ntrajs16/./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00000430
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_independent_ntrajs16/./data/method_comparison_dicke/T5_independent_ntrajs16/config_rho.00001310
-- evaluate_rho_prop_error:./data/method_comparison_dicke/

-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001220
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001070
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000370
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001530
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000790
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00001130
-- evaluate_rho_prop_error:./data/method_comparison_dicke/T5_cross_ntrajs2/./data/method_comparison_dicke/T5_cross_ntrajs2/config_rho.00000600

.  collect_rho_prop_errors:./data/method_comparison_dicke/T5_rho_ntrajs1/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_dicke/T5_nonherm_ntrajs1/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_dicke/T5_independent_ntrajs1/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_dicke/T5_independent_ntrajs2/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_dicke/T5_independent_ntrajs4/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_dicke/T5_independent_ntrajs8/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_dicke/T5_independent_ntrajs16/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_dicke/T5_independent_ntrajs32/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_dicke/T5_independent_ntrajs64/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_dicke/T5_cross_ntrajs2/rho_prop_error.dat
